In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from koopmanlib.dictionary import PsiNN
from koopmanlib.solver import KoopmanDLSolver

2025-06-09 08:31:22.226813: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-09 08:31:22.242435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-09 08:31:22.260205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-09 08:31:22.265413: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-09 08:31:22.278650: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dim = 6
hidden_dim = 80
input_dim = 6
n_feature = 14
num_epochs = 100
batch_size = 64
n_train = 6660
n_test = 606
learning_rate = 1e-3

In [3]:
X_train = pd.read_csv('robot_X_train.csv', header=None).values
X_test = pd.read_csv('robot_X_test.csv', header=None).values
U_train = pd.read_csv('robot_U_train.csv', header=None).values
U_test = pd.read_csv('robot_U_test.csv', header=None).values

length = X_train.shape[1] // n_train
HX_train = []
HU_train = []
for i in range(n_train):
    HX_train.append(X_train[:, i*length:(i+1)*length])
    HU_train.append(U_train[:, i*length:(i+1)*length])
HX_train = np.stack([HX_train[idx].T for idx in range(n_train)], axis=0)
HU_train = np.stack([HU_train[idx].T for idx in range(n_train)], axis=0)
HX_test = []
HU_test = []
for i in range(n_test):
    HX_test.append(X_test[:, i*length:(i+1)*length])
    HU_test.append(U_test[:, i*length:(i+1)*length])
HX_test = np.stack([HX_test[idx].T for idx in range(n_test)], axis=0)
HU_test = np.stack([HU_test[idx].T for idx in range(n_test)], axis=0)

H_train = np.concatenate([HX_train, HU_train], axis=-1)
H_test = np.concatenate([HX_test, HU_test], axis=-1)

In [ ]:
error_temp = []

data_train = [H_train[:, 0:5,:].reshape(-1, dim+input_dim), H_train[:, 1:6,:].reshape(-1, dim+input_dim)]
data_valid = [H_train[:, 0:5,:].reshape(-1, dim+input_dim), H_train[:, 1:6,:].reshape(-1, dim+input_dim)]
basis_function = PsiNN(layer_sizes=[hidden_dim, hidden_dim, hidden_dim], n_psi_train=dim+n_feature)
solver = KoopmanDLSolver(dic=basis_function,
                        target_dim=dim+input_dim,
                        reg=0.0)
solver.build(data_train=data_train,
                 data_valid=data_valid,
                 epochs=num_epochs,
                 batch_size=batch_size,
                 lr=learning_rate,
                 log_interval=1,
                 lr_decay_factor=0.92)
for j in range(n_test):
    x_traj = H_test[j, 0:6,:]
    x0_test = x_traj[0]
    x0_test = x0_test.reshape(-1, x0_test.shape[-1])
    x_est_traj_DL = solver.predict(x0_test, 5)
    error_temp.append((x_est_traj_DL[:, :dim]-x_traj[1:, :dim]) ** 2)
error = np.mean(np.array(error_temp))
    
df = pd.DataFrame([error])
df.to_csv("Robot_EDMDDL.csv", index=False)